In [1]:
import pandas as pd
import numpy as np
import os 

In [2]:
root_dir = '/nfs/turbo/umms-indikar/temp-sam/added-LTS/Pore-C-Snakemake/results_fsu_cmb/align_table/'
file_flag = 'NlaIII_run11'
ext = 'pore_c.parquet'

df_list = []

for f in os.listdir(root_dir):
    if file_flag in f and f.endswith(ext):
        full_path = f"{root_dir}{f}"
        
        tmp = pd.read_parquet(full_path)
#         print(tmp.shape)
        df_list.append(tmp)
        
        
df = pd.concat(df_list, ignore_index=True)
print(df.shape)
df.head()

(1821083, 28)


,read_idx,align_idx,align_type,chrom,start,end,strand,read_name,read_length,read_start,...,filter_reason,fragment_id,num_contained_fragments,num_overlapping_fragments,overlap_length,fragment_start,fragment_end,perc_of_alignment,perc_of_fragment,is_contained
0,0,0,primary,NC_000083.7,17212759,17213253,True,2d93e99a-4a88-4cf8-be35-07704453241d,1534,533,...,pass,10657355,1,3,486,17212765,17213251,98.380569,100.000000,True
1,0,1,primary,NC_000083.7,17169856,17170188,False,2d93e99a-4a88-4cf8-be35-07704453241d,1534,995,...,pass,10657157,0,2,328,17169860,17170353,98.795181,66.531441,False
2,0,2,primary,NC_000083.7,17156780,17157015,False,2d93e99a-4a88-4cf8-be35-07704453241d,1534,0,...,pass,10657099,0,1,235,17156708,17157687,100.000000,24.004086,False
3,0,3,primary,NC_000083.7,17156704,17156844,True,2d93e99a-4a88-4cf8-be35-07704453241d,1534,1007,...,not_on_shortest_path,10657099,0,2,136,17156708,17157687,97.142860,13.891726,False
4,0,4,primary,NC_000083.7,17170339,17170502,True,2d93e99a-4a88-4cf8-be35-07704453241d,1534,75,...,not_on_shortest_path,10657158,1,3,100,17170353,17170453,61.349693,100.000000,True


In [3]:
"""
STEP 0:

Filter to a single chomosome
"""

CHROM = 'NC_000067.7'

df = df[df['chrom'] == CHROM]
print(f"shape: {df.shape}")
print(f"unique reads: {df['read_name'].nunique()}")
print(f"unique fragments: {df['fragment_id'].nunique()}")

shape: (221406, 28)
unique reads: 46766
unique fragments: 20472


In [4]:
"""
STEP 1:

Drop alignments with mapping quality greater than the lower bound and
less than the upper bound
"""

LOW_MAPQ = 30
HIGH_MAPQ = 60


mask = (df['mapping_quality'] > LOW_MAPQ) & (df['mapping_quality'] < HIGH_MAPQ)
print(f"pre shape: {df.shape}")
df = df[mask]
print(f"post shape: {df.shape}")

pre shape: (221406, 28)
post shape: (67779, 28)


In [5]:
"""
STEP 2:

(1) For each read, count the total number of fragments and the number of unique fragments. 

(2) Drop reads where there are fewer than 2 unique fragments (no possible pairwise contacts)
"""

fragment_stats = df.groupby('read_name', as_index=False).agg(
    fragment_count = ('fragment_id', 'count'),
    unique_fragment_count = ('fragment_id', pd.Series.nunique),
)


multi_fragments = fragment_stats[fragment_stats['unique_fragment_count'] > 1]

# filter the original data table
print(f"pre shape: {df.shape}")
df = df[df['read_name'].isin(multi_fragments['read_name'])]
print(f"post shape: {df.shape}")
print(f"unique reads: {df['read_name'].nunique()}")
print(f"unique fragments: {df['fragment_id'].nunique()}")

pre shape: (67779, 28)
post shape: (49115, 28)
unique reads: 14847
unique fragments: 8514


In [15]:
"""
STEP 3:

(1) For each read, take the unique set of fragment ids

(2) if there are multiple duplicate fragment ids, i.e.,
alignments that map to the same spot on the reference from 
different spots on the read, take the single alignment
with the highest mapping quality. 
"""
# get a list of read_name, fragment_id pairs and their counts and 
# MAPQ scores

dup_frags = df.groupby(['read_name', 'fragment_id'], as_index=False).agg(
    n_fragment = ('read_idx', 'count'),
    max_MAPQ = ('mapping_quality', 'max'),
)
    
# only consider duplicated fragment IDS
dup_frags = dup_frags[dup_frags['n_fragment'] > 1]



dup_frags.head()



(39870, 4)
(7652, 4)


,read_name,fragment_id,n_fragment,max_MAPQ
2,00035a4e-fbc3-40ec-8449-2ebfeac22861,673470,2,43
3,00035a4e-fbc3-40ec-8449-2ebfeac22861,674735,2,49
4,00069e40-9c6c-4703-9ed9-f1ffaffb5bf2,768861,2,44
5,00069e40-9c6c-4703-9ed9-f1ffaffb5bf2,769535,2,40
11,000dc696-6a02-47b6-b49d-d089ec218c06,385920,2,36


In [7]:
sample = np.random.choice(df['read_name'].unique(), 1)[0]

tmp = df[df['read_name'] == sample]
tmp = tmp.sort_values(by='read_start', ascending=True)

cols = [
    'read_start',
    'read_end',
    'fragment_id',
    'mapping_quality'
]

tmp[cols].head()